In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import statsmodels.formula.api as smf

In [2]:
# CAS API Keys
api_key = "85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98"

#base_url = "https://api.census.gov/data/2014/acs/acs1"


In [3]:
# Importing the Data
c = Census(api_key, year=2014)
print(c)


In [4]:
census_data = c.acs5.get(("NAME"      , "COUNTY"     , "B06010_001E", "B01002_001E",
                         "B06009_001E", "B06009_002E", "B06009_003E", "B06009_004E", 
                         "B06009_005E", "B06009_006E", "B02001_001E", "B02001_002E",
                         "B02001_003E", "B02001_004E", "B02001_005E", "B02001_006E",
                         "B02001_007E", "B02001_008E", "B01001_026E", "B01001_002E",
                         "B23025_005E", "B05010_001E", "B01003_001E"),{'for': 'county:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

census_pd.head()


,B01001_002E,B01001_026E,B01002_001E,B01003_001E,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,...,B06009_003E,B06009_004E,B06009_005E,B06009_006E,B06010_001E,B23025_005E,COUNTY,NAME,county,state
0,4913688.0,5060515.0,35.3,9974203.0,9974203.0,5329333.0,832253.0,54409.0,1394349.0,26074.0,...,1347268.0,1728972.0,1281851.0,679412.0,8049317.0,564669.0,037,"Los Angeles County, California",037,06
1,73450.0,79002.0,33.3,152452.0,152452.0,125830.0,5382.0,2287.0,3335.0,254.0,...,22190.0,30369.0,8858.0,3804.0,116777.0,5530.0,039,"Madera County, California",039,06
2,125838.0,130964.0,45.1,256802.0,256802.0,203975.0,6913.0,674.0,14565.0,593.0,...,23040.0,48100.0,58107.0,45233.0,212605.0,9756.0,041,"Marin County, California",041,06
3,9050.0,8896.0,50.2,17946.0,17946.0,15850.0,155.0,598.0,232.0,74.0,...,3826.0,5251.0,2006.0,1050.0,15468.0,1132.0,043,"Mariposa County, California",043,06
4,43979.0,43633.0,42.0,87612.0,87612.0,72141.0,705.0,4169.0,1570.0,186.0,...,16137.0,22575.0,8930.0,4842.0,71774.0,5193.0,045,"Mendocino County, California",045,06


In [5]:
# Column Reordering
census_pd = census_pd.rename(columns={"NAME"        : "county_name"       ,
                                      "COUNTY"      : "county"            , 
                                      "B06010_001E" : "gross_income"      , 
                                      "B01002_001E" : "median_age"        ,
                                      "B06009_001E" : "years_of_education", 
                                      "B06009_002E" : "less_than_hs"      , 
                                      "B06009_003E" : "high_school"       , 
                                      "B06009_004E" : "some_college"      , 
                                      "B06009_005E" : "college"           , 
                                      "B06009_006E" : "grad_school"       , 
                                      "B02001_001E" : "total_race"        , 
                                      "B02001_002E" : "white"             ,
                                      "B02001_003E" : "black"             , 
                                      "B02001_004E" : "native_american"   , 
                                      "B02001_005E" : "asian"             , 
                                      "B02001_006E" : "pacific_islander"  ,
                                      "B02001_007E" : "other"             , 
                                      "B02001_008E" : "multi_racial"      , 
                                      "B01001_026E" : "female"            , 
                                      "B01001_002E" : "male"              ,
                                      "B23025_005E" : "unemployment_rate" ,
                                      "B05010_001E" : "poverty_rate"      , 
                                      "B01003_001E" : "population_size"   })

census_pd.head()

,male,female,median_age,population_size,total_race,white,black,native_american,asian,pacific_islander,...,high_school,some_college,college,grad_school,gross_income,unemployment_rate,county,county_name,county,state
0,4913688.0,5060515.0,35.3,9974203.0,9974203.0,5329333.0,832253.0,54409.0,1394349.0,26074.0,...,1347268.0,1728972.0,1281851.0,679412.0,8049317.0,564669.0,037,"Los Angeles County, California",037,06
1,73450.0,79002.0,33.3,152452.0,152452.0,125830.0,5382.0,2287.0,3335.0,254.0,...,22190.0,30369.0,8858.0,3804.0,116777.0,5530.0,039,"Madera County, California",039,06
2,125838.0,130964.0,45.1,256802.0,256802.0,203975.0,6913.0,674.0,14565.0,593.0,...,23040.0,48100.0,58107.0,45233.0,212605.0,9756.0,041,"Marin County, California",041,06
3,9050.0,8896.0,50.2,17946.0,17946.0,15850.0,155.0,598.0,232.0,74.0,...,3826.0,5251.0,2006.0,1050.0,15468.0,1132.0,043,"Mariposa County, California",043,06
4,43979.0,43633.0,42.0,87612.0,87612.0,72141.0,705.0,4169.0,1570.0,186.0,...,16137.0,22575.0,8930.0,4842.0,71774.0,5193.0,045,"Mendocino County, California",045,06


In [6]:
# Values per capita

census_pd["gross_income_pc"]       = census_pd["gross_income"] / census_pd["population_size"] 
census_pd["years_of_education_pc"] = census_pd["years_of_education"] / census_pd["population_size"] 
census_pd["less_than_hs"]          = census_pd["less_than_hs"] / census_pd["population_size"] 
census_pd["high_school_pc"]        = census_pd["high_school"] / census_pd["population_size"] 
census_pd["some_college_pc"]       = census_pd["some_college"] / census_pd["population_size"] 
census_pd["college_pc"]            = census_pd["college"] / census_pd["population_size"] 
census_pd["grad_school_pc"]        = census_pd["grad_school"] / census_pd["population_size"] 
census_pd["total_race_pc"]         = census_pd["total_race"] / census_pd["population_size"] 
census_pd["white_pc"]              = census_pd["white"] / census_pd["population_size"]
census_pd["black_pc"]              = census_pd["black"] / census_pd["population_size"]
census_pd["native_american_pc"]    = census_pd["native_american"] / census_pd["population_size"]
census_pd["asian_pc"]              = census_pd["asian"] / census_pd["population_size"]
census_pd["pacific_islander_pc"]   = census_pd["pacific_islander"]/ census_pd["population_size"]
census_pd["other_pc"]              = census_pd["other"] / census_pd["population_size"]
census_pd["multi_racial_pc"]       = census_pd["multi_racial"] / census_pd["population_size"]
census_pd["female_pc"]             = census_pd["female"] / census_pd["population_size"]
census_pd["male_pc"]               = census_pd["male"] / census_pd["population_size"]
census_pd["unemployment_rate_pc"]  = census_pd["unemployment_rate"] / census_pd["population_size"]
census_pd["poverty_rate_pc"]       = census_pd["poverty_rate"] / census_pd["population_size"]

census_pd.head()

,male,female,median_age,population_size,total_race,white,black,native_american,asian,pacific_islander,...,black_pc,native_american_pc,asian_pc,pacific_islander_pc,other_pc,multi_racial_pc,female_pc,male_pc,unemployment_rate_pc,poverty_rate_pc
0,4913688.0,5060515.0,35.3,9974203.0,9974203.0,5329333.0,832253.0,54409.0,1394349.0,26074.0,...,0.083441,0.005455,0.139796,0.002614,0.195498,0.038885,0.507360,0.492640,0.056613,0.224347
1,73450.0,79002.0,33.3,152452.0,152452.0,125830.0,5382.0,2287.0,3335.0,254.0,...,0.035303,0.015001,0.021876,0.001666,0.067261,0.033519,0.518209,0.481791,0.036274,0.267205
2,125838.0,130964.0,45.1,256802.0,256802.0,203975.0,6913.0,674.0,14565.0,593.0,...,0.026920,0.002625,0.056717,0.002309,0.076573,0.040568,0.509980,0.490020,0.037990,0.199909
3,9050.0,8896.0,50.2,17946.0,17946.0,15850.0,155.0,598.0,232.0,74.0,...,0.008637,0.033322,0.012928,0.004123,0.022233,0.035551,0.495709,0.504291,0.063078,0.159200
4,43979.0,43633.0,42.0,87612.0,87612.0,72141.0,705.0,4169.0,1570.0,186.0,...,0.008047,0.047585,0.017920,0.002123,0.063188,0.037723,0.498025,0.501975,0.059273,0.199801


In [19]:
#CoolClimate Data
climate_data_to_load = "climate_data_county_ajc.csv"
climate_pd = pd.read_csv(climate_data_to_load)
climate_pd.head()

Index(['Population', 'State', 'County', 'electricity (kWh)',
       'Nat. Gas (cu.ft.)', 'FUELOIL (gallons)', 'Vehicle miles traveled',
       ' Transport (tCO2e/yr) ', ' Housing (tCO2e/yr) ', ' Food (tCO2e/yr) ',
       ' Goods (tCO2e/yr) ', ' Services (tCO2e/yr) ',
       ' Total Household Carbon Footprint (tCO2e/yr) ', ' Households ',
       ' Total County Carbon Footprint (tCO2e/yr) '],
      dtype='object')

In [23]:
# County Area
climate_data_to_load_zip_level = "zip_code_clumate_data.csv"
climate_pd_zip_level = pd.read_csv(climate_data_to_load_zip_level)
climate_pd_zip_level.head()
climate_pd_zip_level.columns

Index(['ZipCode', 'Population', 'PersonsPerHousehold', 'AverageHouseValue',
       'IncomePerHousehold', 'Latitude', 'Longitude', 'Elevation', 'State',
       'StateFullName', 'City', 'CountyName', 'CBSAType', 'CBSAName',
       'MSAName', 'PMSA_Name', 'Region', 'Division', ' popden ',
       'electricity (kWh)', 'Nat. Gas (cu.ft.)', 'FUELOIL (gallons)',
       ' Vehicle miles traveled ', ' Transport (tCO2e/yr) ',
       ' Housing (tCO2e/yr) ', ' Food (tCO2e/yr) ', ' Goods (tCO2e/yr) ',
       ' Services (tCO2e/yr) ',
       ' Total Household Carbon Footprint (tCO2e/yr) ', 'HouseholdsPerZipCode',
       'Total Zip Code Carbon Footprint (tCO2e/yr)', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41'],
      dtype='object')

In [45]:
surface_area = climate_pd_zip_level[['Population', ' popden ', 'CountyName']]
surface_area['popden'] = surface_area[' popden '].str.replace(',', '')
surface_area['popden'] = surface_area['popden'].astype(float)
surface_area.head()
surface_area['surface_area'] = surface_area['Population'] / surface_area['popden']
surface_area = surface_area.groupby(['CountyName']).sum().reset_index()
surface_area = surface_area[['CountyName', 'surface_area']]
surface_area 

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,CountyName,surface_area
0,ABBEVILLE,430.659713
1,ACADIA,598.346869
2,ACCOMACK,453.870613
3,ADA,963.427756
4,ADAIR,1949.714678
5,ADAMS,8545.160785
6,ADDISON,771.029665
7,AIKEN,857.992634
8,AITKIN,1369.320833
9,ALACHUA,1077.141158


In [12]:
# Data with County and State
us_states_to_load = "us_states.csv"

us_states_df = pd.read_csv(us_states_to_load, 
                           header = None)

us_states_df.columns = ["Upper Full Name", 
                        "Proper Full Name", 
                        "State"]
us_states_df.head()

,Upper Full Name,Proper Full Name,State
0,ALABAMA,Alabama,AL
1,ALASKA,Alaska,AK
2,ARIZONA,Arizona,AZ
3,ARKANSAS,Arkansas,AR
4,CALIFORNIA,California,CA


In [47]:
# Merge both dataframes
states_complete = pd.merge(us_states_df, 
                           climate_pd, 
                           how = "left", 
                           on = ["State"]) 

states_complete = pd.merge(states_complete, 
                          surface_area,
                          how = "left",
                          left_on = 'Upper Full Name',
                          right_on = 'CountyName')
states_complete


,Upper Full Name,Proper Full Name,State,Population,County,electricity (kWh),Nat. Gas (cu.ft.),FUELOIL (gallons),Vehicle miles traveled,Transport (tCO2e/yr),Housing (tCO2e/yr),Food (tCO2e/yr),Goods (tCO2e/yr),Services (tCO2e/yr),Total Household Carbon Footprint (tCO2e/yr),Households,Total County Carbon Footprint (tCO2e/yr),CountyName,surface_area
0,ALABAMA,Alabama,AL,14387,SUMTER,"12,667","28,415",6,"18,175",11.9,13.9,7.7,3.7,3.8,41.0,6654.0,"272,685",NaN,NaN
1,ALABAMA,Alabama,AL,13563,WILCOX,"12,846","26,873",10,"18,756",12.2,14.1,8.2,3.7,3.7,41.8,6293.0,"262,872",NaN,NaN
2,ALABAMA,Alabama,AL,22987,MACON,"12,983","34,664",1,"18,269",12.1,14.6,7.3,3.9,4.0,41.9,10104.0,"422,969",NaN,NaN
3,ALABAMA,Alabama,AL,9809,GREENE,"13,157","29,872",4,"18,821",12.4,14.4,7.5,3.7,3.9,41.9,5029.0,"210,619",NaN,NaN
4,ALABAMA,Alabama,AL,12701,BULLOCK,"12,493","28,330",1,"19,187",12.6,13.8,7.7,3.9,4.0,41.9,5166.0,"216,678",NaN,NaN
5,ALABAMA,Alabama,AL,9830,PERRY,"13,880","32,364",3,"18,226",11.9,15.1,8.0,3.7,3.8,42.5,4266.0,"181,153",NaN,NaN
6,ALABAMA,Alabama,AL,30485,PIKE,"11,983","27,026",3,"21,059",14.0,13.3,7.2,4.1,4.3,42.8,14435.0,"618,425",NaN,NaN
7,ALABAMA,Alabama,AL,27630,BARBOUR,"12,508","27,275",1,"20,836",13.8,13.8,7.6,4.1,4.3,43.5,11744.0,"511,117",NaN,NaN
8,ALABAMA,Alabama,AL,48892,RUSSELL,"12,328","28,993",2,"20,674",13.8,13.7,7.5,4.3,4.6,43.9,26673.0,"1,171,923",NaN,NaN
9,ALABAMA,Alabama,AL,13892,CONECUH,"13,814","28,256",1,"21,171",13.9,14.8,7.3,3.9,4.1,44.0,7000.0,"307,713",NaN,NaN


In [48]:
# Getting variables in the correct format
states_complete["County"] = states_complete["County"].str.title()
states_complete.head()

,Upper Full Name,Proper Full Name,State,Population,County,electricity (kWh),Nat. Gas (cu.ft.),FUELOIL (gallons),Vehicle miles traveled,Transport (tCO2e/yr),Housing (tCO2e/yr),Food (tCO2e/yr),Goods (tCO2e/yr),Services (tCO2e/yr),Total Household Carbon Footprint (tCO2e/yr),Households,Total County Carbon Footprint (tCO2e/yr),CountyName,surface_area
0,ALABAMA,Alabama,AL,14387,Sumter,"12,667","28,415",6,"18,175",11.9,13.9,7.7,3.7,3.8,41.0,6654.0,"272,685",NaN,NaN
1,ALABAMA,Alabama,AL,13563,Wilcox,"12,846","26,873",10,"18,756",12.2,14.1,8.2,3.7,3.7,41.8,6293.0,"262,872",NaN,NaN
2,ALABAMA,Alabama,AL,22987,Macon,"12,983","34,664",1,"18,269",12.1,14.6,7.3,3.9,4.0,41.9,10104.0,"422,969",NaN,NaN
3,ALABAMA,Alabama,AL,9809,Greene,"13,157","29,872",4,"18,821",12.4,14.4,7.5,3.7,3.9,41.9,5029.0,"210,619",NaN,NaN
4,ALABAMA,Alabama,AL,12701,Bullock,"12,493","28,330",1,"19,187",12.6,13.8,7.7,3.9,4.0,41.9,5166.0,"216,678",NaN,NaN


In [49]:
# Creating the keys to merge the data sets
states_complete["county_name"] = states_complete["County"] + "County, " + states_complete["Proper Full Name"]
states_complete.head()


,Upper Full Name,Proper Full Name,State,Population,County,electricity (kWh),Nat. Gas (cu.ft.),FUELOIL (gallons),Vehicle miles traveled,Transport (tCO2e/yr),Housing (tCO2e/yr),Food (tCO2e/yr),Goods (tCO2e/yr),Services (tCO2e/yr),Total Household Carbon Footprint (tCO2e/yr),Households,Total County Carbon Footprint (tCO2e/yr),CountyName,surface_area,county_name
0,ALABAMA,Alabama,AL,14387,Sumter,"12,667","28,415",6,"18,175",11.9,13.9,7.7,3.7,3.8,41.0,6654.0,"272,685",NaN,NaN,"Sumter County, Alabama"
1,ALABAMA,Alabama,AL,13563,Wilcox,"12,846","26,873",10,"18,756",12.2,14.1,8.2,3.7,3.7,41.8,6293.0,"262,872",NaN,NaN,"Wilcox County, Alabama"
2,ALABAMA,Alabama,AL,22987,Macon,"12,983","34,664",1,"18,269",12.1,14.6,7.3,3.9,4.0,41.9,10104.0,"422,969",NaN,NaN,"Macon County, Alabama"
3,ALABAMA,Alabama,AL,9809,Greene,"13,157","29,872",4,"18,821",12.4,14.4,7.5,3.7,3.9,41.9,5029.0,"210,619",NaN,NaN,"Greene County, Alabama"
4,ALABAMA,Alabama,AL,12701,Bullock,"12,493","28,330",1,"19,187",12.6,13.8,7.7,3.9,4.0,41.9,5166.0,"216,678",NaN,NaN,"Bullock County, Alabama"


In [50]:
# Renaming some variables
states_complete = states_complete.rename(columns={"Upper Full Name"                              : "upper_state"       ,
                                                  "Proper Full Name"                             : "title_state"       ,
                                                  "State"                                        : "state_abbrev"      ,
                                                  "Population"                                   : "population"        ,
                                                  "County"                                       : "county"            ,
                                                  "electricity (kWh)"                            : "electricity"       ,
                                                  "Nat. Gas (cu.ft.)"                            : "natural_gas"       ,
                                                  "FUELOIL (gallons)"                            : "fuel_oil"          ,
                                                  "Vehicle miles traveled"                       : "vehicle_miles"     ,
                                                  " Transport (tCO2e/yr) "                       : "transport_fp"      ,
                                                  " Housing (tCO2e/yr) "                         : "housing_fp"        ,
                                                  " Food (tCO2e/yr) "                            : "food_fp"           ,
                                                  " Goods (tCO2e/yr) "                           : "goods_fp"          ,
                                                  " Services (tCO2e/yr) "                        : "services_fp"       ,
                                                  " Total Household Carbon Footprint (tCO2e/yr) ": "household_total_fp",
                                                  " Households "                                 : "household_count"   ,
                                                  " Total County Carbon Footprint (tCO2e/yr) "   : "county_total_fp"}  )
states_complete.head()

,upper_state,title_state,state_abbrev,population,county,electricity,natural_gas,fuel_oil,vehicle_miles,transport_fp,housing_fp,food_fp,goods_fp,services_fp,household_total_fp,household_count,county_total_fp,CountyName,surface_area,county_name
0,ALABAMA,Alabama,AL,14387,Sumter,"12,667","28,415",6,"18,175",11.9,13.9,7.7,3.7,3.8,41.0,6654.0,"272,685",NaN,NaN,"Sumter County, Alabama"
1,ALABAMA,Alabama,AL,13563,Wilcox,"12,846","26,873",10,"18,756",12.2,14.1,8.2,3.7,3.7,41.8,6293.0,"262,872",NaN,NaN,"Wilcox County, Alabama"
2,ALABAMA,Alabama,AL,22987,Macon,"12,983","34,664",1,"18,269",12.1,14.6,7.3,3.9,4.0,41.9,10104.0,"422,969",NaN,NaN,"Macon County, Alabama"
3,ALABAMA,Alabama,AL,9809,Greene,"13,157","29,872",4,"18,821",12.4,14.4,7.5,3.7,3.9,41.9,5029.0,"210,619",NaN,NaN,"Greene County, Alabama"
4,ALABAMA,Alabama,AL,12701,Bullock,"12,493","28,330",1,"19,187",12.6,13.8,7.7,3.9,4.0,41.9,5166.0,"216,678",NaN,NaN,"Bullock County, Alabama"


In [51]:
climate_census = pd.merge(census_pd,
                          states_complete,
                          how = "inner",
                          on = "county_name")
climate_census.columns

Index(['male', 'female', 'median_age', 'population_size', 'total_race',
       'white', 'black', 'native_american', 'asian', 'pacific_islander',
       'other', 'multi_racial', 'poverty_rate', 'years_of_education',
       'less_than_hs', 'high_school', 'some_college', 'college', 'grad_school',
       'gross_income', 'unemployment_rate', 'county_x', 'county_name',
       'county_x', 'state', 'gross_income_pc', 'years_of_education_pc',
       'high_school_pc', 'some_college_pc', 'college_pc', 'grad_school_pc',
       'total_race_pc', 'white_pc', 'black_pc', 'native_american_pc',
       'asian_pc', 'pacific_islander_pc', 'other_pc', 'multi_racial_pc',
       'female_pc', 'male_pc', 'unemployment_rate_pc', 'poverty_rate_pc',
       'upper_state', 'title_state', 'state_abbrev', 'population', 'county_y',
       'electricity', 'natural_gas', 'fuel_oil', 'vehicle_miles',
       'transport_fp', 'housing_fp', 'food_fp', 'goods_fp', 'services_fp',
       'household_total_fp', 'household_count', '

In [53]:
climate_census["pop_den"] = climate_census["population_size"] / climate_census["surface_area"]  

In [52]:
variables_to_clean = ['electricity'  , 'natural_gas', 'fuel_oil'       , 
                      'vehicle_miles', 'housing_fp' , 'county_total_fp']

for variable in variables_to_clean:
    climate_census[variable] = climate_census[variable].str.replace(' ', '')
    climate_census[variable] = climate_census[variable].str.replace('-', '')
    climate_census[variable] = climate_census[variable].str.replace(',', '')

    

In [58]:
#Regression with robustcov
# Regressions on transportation
lm_transport = smf.ols(formula = 'transport_fp ~ gross_income_pc + high_school_pc + some_college_pc + college_pc + grad_school_pc + white_pc + black_pc + native_american_pc + asian_pc + pacific_islander_pc + other_pc + multi_racial_pc + female_pc + male_pc + unemployment_rate_pc + poverty_rate_pc + pop_den', 
                       data = climate_census).fit().get_robustcov_results()
#print(lm_transport.params) 
lm_transport_summary = lm_transport.summary()
lm_transport_summary

/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 17, but rank is 16
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           transport_fp   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.468
Method:                 Least Squares   F-statistic:                     8194.
Date:                Sat, 24 Aug 2019   Prob (F-statistic):               0.00
Time:                        10:57:16   Log-Likelihood:                -2057.7
No. Observations:                1072   AIC:                             4147.
Df Residuals:                    1056   BIC:                             4227.
Df Model:                          15                                         
Covariance Type:                  HC1                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              -11.2008      6.621     -1.692      0.091     -24.192       1.790
gross_income_pc         12.3500     11.089      1.114      0.266      -9.409      34.109
high_school_pc          15.0669      1.825      8.258      0.000      11.487      18.647
some_college_pc         13.0576      1.838      7.103      0.000       9.451      16.665
college_pc              20.7380      3.613      5.739      0.000      13.648      27.828
grad_school_pc          -3.7176      5.029     -0.739      0.460     -13.586       6.150
white_pc                 5.8740      1.918      3.062      0.002       2.110       9.638
black_pc                 1.7313      1.866      0.928      0.354      -1.931       5.394
native_american_pc       2.6462      2.312      1.145      0.253      -1.890       7.182
asian_pc                16.5260      5.255      3.145      0.002       6.214      26.838
pacific_islander_pc    -46.8455     13.839     -3.385      0.001     -74.001     -19.690
other_pc                 2.9881      2.205      1.355      0.176      -1.339       7.315
multi_racial_pc          5.8792      3.798      1.548      0.122      -1.573      13.332
female_pc              -11.4637      3.495     -3.280      0.001     -18.322      -4.606
male_pc                  0.2629      3.578      0.073      0.941      -6.758       7.284
unemployment_rate_pc    -4.0893      4.991     -0.819      0.413     -13.883       5.705
poverty_rate_pc         46.7525      9.446      4.950      0.000      28.218      65.287
pop_den              -7.084e-05   1.41e-05     -5.020      0.000   -9.85e-05   -4.32e-05
==============================================================================
Omnibus:                       65.632   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              186.514
Skew:                           0.279   Prob(JB):                     3.15e-41
Kurtosis:                       4.966   Cond. No.                     1.01e+20
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
[2] The smallest eigenvalue is 4.61e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [54]:


#Regression
# Regressions on  housing
lm_housing = smf.ols(formula = 'housing_fp ~ gross_income_pc + high_school_pc + some_college_pc + college_pc + grad_school_pc + white_pc + black_pc + native_american_pc + asian_pc + pacific_islander_pc + other_pc + multi_racial_pc + female_pc + male_pc + unemployment_rate_pc + poverty_rate_pc', data = climate_census).fit().get_robustcov_results()
#print(lm_housing.params) 
lm_housing_summary = lm_housing.summary()

#Regression
# Regressions on goods 
lm_goods = smf.ols(formula = 'goods_fp ~ gross_income_pc + high_school_pc + some_college_pc + college_pc + grad_school_pc + white_pc + black_pc + native_american_pc + asian_pc + pacific_islander_pc + other_pc + multi_racial_pc + female_pc + male_pc + unemployment_rate_pc + poverty_rate_pc', data = climate_census).fit().get_robustcov_results()
#print(lm_goods.params) 
lm_goods_summary = lm_goods.summary()

#Regression
# Regressions on services 
lm_services = smf.ols(formula = 'services_fp ~ gross_income_pc + high_school_pc + some_college_pc + college_pc + grad_school_pc + white_pc + black_pc + native_american_pc + asian_pc + pacific_islander_pc + other_pc + multi_racial_pc + female_pc + male_pc + unemployment_rate_pc + poverty_rate_pc', data = climate_census).fit().get_robustcov_results()
#print(lm_services.params) 
lm_services_summary = lm_services.summary()

/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 17, but rank is 16
  'rank is %d' % (J, J_), ValueWarning)


ValueError: shapes (2921,152) and (2921,152) not aligned: 152 (dim 1) != 2921 (dim 0)